In [1]:
import subprocess
import requests

In [2]:
%%file klasa_perceptron.py
import numpy as np
import pandas as pd
class Perceptron:
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
    
    def predict(self, X):
        return np.where(self._net_input(X) >= 0, 1, -1)
    
    def _net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

Overwriting klasa_perceptron.py


In [3]:
%%file model.py
from klasa_perceptron import Perceptron
import pickle 
import pandas as pd 
import numpy as np

from sklearn.datasets import load_iris

iris = load_iris()
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], 
                  columns=iris['feature_names']+['target'])
X = df.iloc[:100,[0,2]].values
y = df.iloc[:100,4].values

model = Perceptron()   
model.fit(X, y)  
model.errors_
model.w_
model.predict(X)

#save model to file
with open('perceptron_model.pkl', 'wb') as pp:
    pickle.dump(model, pp)

Overwriting model.py


In [4]:
%run model.py

In [5]:
%%file app.py

import numpy as np
from flask import Flask, request, jsonify
import pickle
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String


engine = create_engine('sqlite:///perceptron_predict.db')

SQLALCHEMY_TRACK_MODIFICATIONS = False


# Create a flask
app = Flask(__name__)

app.config ['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///perceptron_predict.db'

mmodel = pickle.load(open('perceptron_model.pkl','rb'))

# create table named Results in created database
meta = MetaData()
results = Table(
   'Results', meta, 
   Column('id', Integer, primary_key = True), 
   Column('SepalLength', Integer), 
   Column('PetalLength', Integer),
   Column('ClassificationResult', Integer),
)


# Create an API end point
@app.route('/api', methods=['GET'])
def classification():
    sepal_length = float(request.args.get("sepal_length", ""))
    petal_length = float(request.args.get("petal_length", ""))
    lista = [sepal_length, petal_length]
    weights = mmodel.w_

    if sepal_length>0 and petal_length>0:
#generate classification result based on inserted values
        target = mmodel.predict([lista])
#creating json file returned by function
        resp = jsonify(SepalLength=sepal_length, 
                       PetalLength=petal_length, 
                       Weights=list(weights),
                       ClassificationResult = target.tolist())
# establish connection with our database
        conn = engine.connect()
# inserting inserted values and classification result to table Results
        ins =results.insert().values(SepalLength=sepal_length, 
                       PetalLength=petal_length,
                       ClassificationResult = target.tolist()[0])
        result = conn.execute(ins)
    else:
        resp = "Wprowadzono nieprawidlowe wartosci"
    return resp



if __name__ == '__main__':
    meta.create_all(engine)
    app.run(port=4917, debug=True)

Overwriting app.py


In [6]:
p = subprocess.Popen(["python", "app.py"])

 * Serving Flask app 'app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:4917 (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 753-429-189


In [7]:
response = requests.get("http://127.0.0.1:4917/api?sepal_length=5.1&petal_length=1.4")
response.text

127.0.0.1 - - [06/May/2022 22:31:04] "GET /api?sepal_length=5.1&petal_length=1.4 HTTP/1.1" 200 -


'{\n  "ClassificationResult": [\n    -1\n  ], \n  "PetalLength": 1.4, \n  "SepalLength": 5.1, \n  "Weights": [\n    0.0, \n    -0.016000000000000056, \n    0.043999999999999956\n  ]\n}\n'

In [8]:
p.kill()